In [18]:
import os
import sys
import logging

from dotenv import load_dotenv
from llama_index.core.agent import ReActAgent
from llama_index.core.callbacks import CallbackManager
from llama_index.core.tools.types import BaseTool
from llama_index.tools.tavily_research import TavilyToolSpec
from llama_index.llms.bedrock_converse import BedrockConverse
from langfuse.llama_index import LlamaIndexCallbackHandler

In [12]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(formatter)
logger.addHandler(handler)

load_dotenv()

True

In [13]:
MODEL_ID=os.environ.get("MODEL_ID", "anthropic.claude-3-haiku-20240307-v1:0")
AWS_PROFILE_NAME=os.environ.get("AWS_PROFILE_NAME", None)
TAVILY_API_KEY=os.environ.get("TAVILY_API_KEY", None)
LANGFUSE_PUBLIC_KEY=os.environ.get("LANGFUSE_PUBLIC_KEY", None)
LANGFUSE_SECRET_KEY=os.environ.get("LANGFUSE_SECRET_KEY", None)
LANGFUSE_HOST=os.environ.get("LANGFUSE_HOST", None)

assert TAVILY_API_KEY

In [15]:
# setup tools
tools: list[BaseTool] = [
    TavilyToolSpec(api_key=TAVILY_API_KEY).to_tool_list()[0],
]

# setup llm
llm = BedrockConverse(
    model=MODEL_ID,
    max_tokens=1024 * 2,
    temperature=0.4,
    profile_name=AWS_PROFILE_NAME,
)

# setup tracing
langfuse_callback_handler = LlamaIndexCallbackHandler(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_HOST,
)
callback_manager = CallbackManager([langfuse_callback_handler])

In [16]:
# setup agent
agent = ReActAgent.from_tools(
    tools=tools,
    llm=llm,
    callback_manager=callback_manager,
    verbose=True,
)

In [17]:
resp = agent.chat(
    "NCSoft 의 최신작과 월드 오브 워크래프트의 차이가 뭐야?"
    "캐주얼한 게이머에게 어떤 게임이 더 좋을까?"
)
logger.info(f"=== Response ===\n{resp}")

> Running step f2060762-f6a3-4964-b1aa-98339cd7b99a. Step input: NCSoft 의 최신작과 월드 오브 워크래프트의 차이가 뭐야?캐주얼한 게이머에게 어떤 게임이 더 좋을까?
Thought: The current language of the user is: Korean. I need to use a tool to help me answer the question.
Action: search
Action Input: {'query': 'NCSoft 최신작 vs 월드 오브 워크래프트 차이', 'max_results': 3}
Observation: [Document(id_='034490e5-f8c1-4c30-b678-244c3163c962', embedding=None, metadata={'url': 'https://www.etoday.co.kr/news/view/1792469'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='월드 오브 워크래프트(이하 와우) 클래식이 대중 앞에 모습을 드러냈다. 이에 오리지널 버전과 달라진 점을 짚어봤다.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='bfd11f5a-015f-46f3-b376-1e759b61e487', embedding=None, metadata={'url': 'https://namu.wiki/w/월드+오브+워크래프트+클래식'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, 

Unexpected error occurred. Please check your request and contact support: https://langfuse.com/support.
